In [1]:
from erddapy import ERDDAP
import folium
import pandas as pd


Nowcast - Straight of Juan de Fuca

In [2]:
server = "http://coastwatch.pfeg.noaa.gov/erddap"

e = ERDDAP(server=server, protocol="tabledap")

e.dataset_id = "cwwcNDBCMet"

e.variables = [
    "station",
    "time",
    "latitude",
    "longitude",
    "wvht",
    "dpd",
    "mwd",
    "wd",
    "wspd",
    "wtmp"
]

e.constraints = {"time>=":"2023-08-01T00:00:00Z",
                "station=":"46087"}

df = e.to_pandas()

recent = df.dropna().sort_values('time (UTC)').tail(1)

In [3]:
recent

,station,time (UTC),latitude (degrees_north),longitude (degrees_east),wvht (m),dpd (s),mwd (degrees_true),wd (degrees_true),wspd (m s-1),wtmp (degree_C)
1033,46087,2023-08-09T23:50:00Z,48.494,-124.727,1.4,9.0,278.0,210.0,3.0,11.9


In [4]:
if (340.0 > recent['mwd (degrees_true)'].values[0] > 260.0) :
    print("The swell is getting into the straight")
    
    if (270 > recent['wd (degrees_true)'].values[0] > 90) :
        print("The wind is offshore for north facing spots")
    elif ((recent['wd (degrees_true)'].values[0] > 210.0) or (recent['wd (degrees_true)'].values[0] < 120.0)) :
        print("wind is offshore for south facers")
else :
    print("The swell isn't getting into the straight; conditions will be flat")


The swell is getting into the straight
The wind is offshore for north facing spots


Nowcast - Westport/Grays Harbor

In [5]:
e.constraints = {"time>=":"2023-08-01T00:00:00Z",
                "station=":"46211"}

df = e.to_pandas()

recent = df.sort_values('time (UTC)').tail(1)

In [6]:
recent

,station,time (UTC),latitude (degrees_north),longitude (degrees_east),wvht (m),dpd (s),mwd (degrees_true),wd (degrees_true),wspd (m s-1),wtmp (degree_C)
429,46211,2023-08-09T23:56:00Z,46.857,-124.244,1.2,8.0,275,NaN,NaN,17.6


In [7]:
spots = pd.DataFrame({
    "name":["Jordan River", "Crescent Beach", "Westport", "Fort Ebey"],
    "lon":[-124.0562, -123.7183, -124.1496, -122.7736],
    "lat":[48.4195, 48.1651, 46.9009, 48.2198],
    "min_wd":[270, 90, 0, 0],
    "max_wd":[90, 270, 180, 180],
    "min_sd":[260, 260, 210, 260],
    "max_sd":[340, 240, 330, 340]})

spots

,name,lon,lat,min_wd,max_wd,min_sd,max_sd
0,Jordan River,-124.0562,48.4195,270,90,260,340
1,Crescent Beach,-123.7183,48.1651,90,270,260,240
2,Westport,-124.1496,46.9009,0,180,210,330
3,Fort Ebey,-122.7736,48.2198,0,180,260,340


In [8]:
min_lon, max_lon = -126, -122
min_lat, max_lat = 48, 49

m = folium.Map(
    location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2],
    zoom_start=9,
)

m